# Import

In [1]:
import os
import cv2
import csv
import math
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold

# Global variable

In [2]:
# PIDROOT_WALK = "../1_GaitAnalysis_ver1/data/"
# PIDROOT_SIT = "../1_GaitAnalysis_ver1/data/"

PIDROOT_GAIT = "../4_Cut_Video/data/" ### [update] 2023/08/26
PIDROOT_EYES = "../2_EyeTrackingAnalysis/data/"

Turn_remove_list = []
Eyes_remove_list = []

n_FOLD = 5

# Load Label

In [3]:
# excel_path = '步態收案名單_0617.xlsx' # [update] 2023/06/17
excel_path = '步態收案名單_0816.xlsx' # [update] 2023/08/26

df_label = pd.read_excel(excel_path, usecols=['編號', 'Age', 'Gender', '_Label']) # [update] 2023/06/17
df_label = df_label.fillna(0)
df_label = df_label.astype({"_Label": int})

Label_arr = np.array(df_label['_Label'])
Label_dict = dict(zip(df_label.編號, df_label._Label))

print(f'len(Label_dict): {len(Label_dict)}\n')
print(Label_dict)

len(Label_dict): 100

{1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 0, 42: 0, 43: 0, 44: 0, 45: 1, 46: 1, 47: 0, 48: 1, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 1, 60: 1, 61: 1, 62: 1, 63: 0, 64: 0, 65: 1, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0}


# Load Id

## Walk & Sit-and-Stand

In [4]:
pid_walk = []
pid_sitstand = []

def load_base_pid(dataPath): ### Walking & Sit-Stand Subtask (88 subjects)
    tmp_list = []
    
    print(f'[Info] Load pid for Walking Subtask & Sit-Stand Subtask ...')
    for pid in list(os.listdir(dataPath)):
        tmp_list.append(pid)
    print(f'- number of subjects: {len(tmp_list)}')
    
    return tmp_list, tmp_list
    
pid_walk, pid_sitstand = load_base_pid(PIDROOT_GAIT)

[Info] Load pid for Walking Subtask & Sit-Stand Subtask ...
- number of subjects: 88


In [5]:
print(f'Walking Subtask pid: {len(pid_walk)} subjects.\n {pid_walk}\n')
print(f'Sit-Stand Subtask pid: {len(pid_sitstand)} subjects.\n {pid_sitstand}')

Walking Subtask pid: 88 subjects.
 ['01', '02', '03', '04', '05', '06', '07', '09', '10', '100', '11', '12', '13', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '43', '44', '45', '46', '48', '49', '51', '52', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']

Sit-Stand Subtask pid: 88 subjects.
 ['01', '02', '03', '04', '05', '06', '07', '09', '10', '100', '11', '12', '13', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '43', '44', '45', '46', '48', '49', '51', '52', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',

## Turning

In [6]:
pid_turn = []
Turn_remove_list = ['04', '07', '09', '32', '60']

def load_turn_pid(walkList):
    tmp_list = []
    
    print(f'[Info] Load pid for Turning Subtask ...')
    for pid in walkList:
        if pid not in Turn_remove_list:
            tmp_list.append(pid)
    print(f'- number of subjects: {len(tmp_list)}\n')
    return tmp_list

pid_turn = load_turn_pid(pid_walk)
print(f'Turning Subtask pid: {len(pid_turn)} subjects.\n {pid_turn}')

[Info] Load pid for Turning Subtask ...
- number of subjects: 83

Turning Subtask pid: 83 subjects.
 ['01', '02', '03', '05', '06', '10', '100', '11', '12', '13', '16', '17', '18', '19', '20', '21', '22', '24', '25', '26', '27', '28', '29', '30', '31', '33', '34', '35', '36', '37', '38', '39', '40', '43', '44', '45', '46', '48', '49', '51', '52', '57', '58', '59', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


## Facial

In [7]:
pid_eyes = []
Eyes_noEyes_list = []
Eyes_noWalk_list = []

# Facial_remove_list = ['10', '11', '60', '61', '85']
Facial_remove_list = ['60', '61', '85'] ### [update] 2023/08/31

def load_eye_pid(dataPath): ### Facial Subtask
    tmp_list = []
    tmp_noWalk = []
    tmp_noEyes = []
    
    print(f'[Info] Load pid for Facial Subtask ...')
    for pid in list(os.listdir(dataPath)):
        if (pid in Facial_remove_list)==False:
            tmp_list.append(pid)
            if pid not in pid_walk:
                tmp_noWalk.append(pid)
    print(f'- number of subjects: {len(tmp_list)}')
    print(f'- number of subjects (only Facial): {len(tmp_noWalk)}')
    print(f'   {tmp_noWalk}')
    
    for pid in pid_walk:
        if pid not in tmp_list:
            tmp_noEyes.append(pid)
    print(f'- number of subjects (no Facial): {len(tmp_noEyes)}')
    print(f'   {tmp_noEyes}')
    
    return tmp_list, tmp_noWalk, tmp_noEyes
    
pid_eyes, Eyes_noWalk_list, Eyes_noEyes_list = load_eye_pid(PIDROOT_EYES)

[Info] Load pid for Facial Subtask ...
- number of subjects: 96
- number of subjects (only Facial): 11
   ['08', '14', '15', '23', '42', '47', '50', '53', '54', '55', '56']
- number of subjects (no Facial): 3
   ['60', '61', '85']


In [8]:
print(f'Facial Subtask pid: {len(pid_eyes)} subjects.\n\n {pid_eyes}\n')
print(f'Only Facial Data (no Gait): {len(Eyes_noWalk_list)} subjects.\n {Eyes_noWalk_list}\n')
print(f'Only Gait Data (no Facial): {len(Eyes_noEyes_list)} subjects.\n {Eyes_noEyes_list}')

Facial Subtask pid: 96 subjects.

 ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']

Only Facial Data (no Gait): 11 subjects.
 ['08', '14', '15', '23', '42', '47', '50', '53', '54', '55', '56']

Only Gait Data (no Facial): 3 subjects.
 ['60', '61', '85']


# Split Id

## Walking

In [9]:
train_walk_5f, val_walk_5f, test_walk_5f = [], [], []
train_turn_5f, val_turn_5f, test_turn_5f = [], [], []
train_sitstand_5f, val_sitstand_5f, test_sitstand_5f = [], [], []
train_eyes_5f, val_eyes_5f, test_eyes_5f = [], [], []

def split_base_5fold(pidList): ### Walking & Sit-Stand Subtask
    tmp_train_5f, tmp_val_5f, tmp_test_5f = [], [], []
    tmp_all_target = []
    
    for pid in pidList:
        pid_int = int(pid)
        tmp_all_target.append(Label_dict[pid_int])

    kf = StratifiedKFold(n_splits=n_FOLD, shuffle=True, random_state=42)

    for train_index, test_index in kf.split(pidList, tmp_all_target):
        realTrain_idx, realTest_idx = [], []
        for idx in train_index:
            realTrain_idx.append(pidList[idx])
        for idx in test_index:
            realTest_idx.append(pidList[idx])
        
        tmp_train_5f.append(realTrain_idx)
        tmp_test_5f.append(realTest_idx)
    
    ### Split Validation Set from Training Set
    tmp_train_5f_new = []
    tmp_val_5f = []
    for i in range(n_FOLD):
        tmp_pidList_1f = tmp_train_5f[i]
        tmp_label_1f = []
        
        for pid in tmp_pidList_1f:
            tmp_label_1f.append(Label_dict[int(pid)])
            
        kf_validation = StratifiedKFold(n_splits=5)
        cnt_split = 0
        for train_index, val_index in kf_validation.split(tmp_pidList_1f, tmp_label_1f):
            train_pid, val_pid = [], []
            if cnt_split==0:
                for idx in train_index:
                    train_pid.append(tmp_pidList_1f[idx])
                for idx in val_index:
                    val_pid.append(tmp_pidList_1f[idx])

                tmp_train_5f_new.append(train_pid)
                tmp_val_5f.append(val_pid)
            cnt_split += 1
        
    return tmp_train_5f_new, tmp_val_5f, tmp_test_5f
    

train_walk_5f, val_walk_5f, test_walk_5f = split_base_5fold(pid_walk)  

In [10]:
print(f'[Info] Show Split result of Walking Subtask ...\n')
for i in range(n_FOLD):
    print(f'[Fold {i+1}] Train: (num: {len(train_walk_5f[i])})\n{train_walk_5f[i]}')
    print(f'[Fold {i+1}] Validate: (num: {len(val_walk_5f[i])})\n{val_walk_5f[i]}')
    print(f'[Fold {i+1}] Test: (num: {len(test_walk_5f[i])})\n{test_walk_5f[i]}\n')    

[Info] Show Split result of Walking Subtask ...

[Fold 1] Train: (num: 56)
['17', '18', '20', '24', '25', '26', '27', '28', '31', '32', '33', '35', '38', '39', '40', '43', '46', '48', '51', '52', '57', '58', '60', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '82', '84', '85', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98', '99']
[Fold 1] Validate: (num: 14)
['03', '05', '06', '07', '09', '10', '100', '12', '13', '16', '19', '21', '22', '30']
[Fold 1] Test: (num: 18)
['01', '02', '04', '11', '29', '34', '36', '37', '44', '45', '49', '59', '67', '75', '81', '83', '89', '94']

[Fold 2] Train: (num: 56)
['17', '18', '20', '24', '25', '26', '27', '28', '30', '31', '32', '33', '34', '36', '37', '38', '39', '40', '44', '45', '46', '48', '49', '51', '57', '58', '59', '60', '63', '64', '67', '68', '69', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '83', '84', '85', '86', '88', '89', '92', '94',

In [11]:
def check_walking_subtask_pid(train_5f, val_5f, test_5f):
    ### check if all subjects are tested
    tmp_set = set()
    for i in range(n_FOLD):
        test_1f = test_5f[i]
        for j in range(len(test_1f)):
            pid = test_1f[j]
            tmp_set.add(pid)
    print(f'Number of pid in Test Set: {len(tmp_set)} subjects.\n')
    
    ### Check each fold (train + val + test)
    print(f'[Info] Check for each fold ...')
    for i in range(n_FOLD):
        tmp_set = set()
        for j in range(len(train_5f[i])):
            tmp_set.add(train_5f[i][j])
        for j in range(len(val_5f[i])):
            tmp_set.add(val_5f[i][j])
        for j in range(len(test_5f[i])):
            tmp_set.add(test_5f[i][j])
        print(f' -fold {i+1}: {len(tmp_set)} subjects.')
    
check_walking_subtask_pid(train_walk_5f, val_walk_5f, test_walk_5f)  

Number of pid in Test Set: 88 subjects.

[Info] Check for each fold ...
 -fold 1: 88 subjects.
 -fold 2: 88 subjects.
 -fold 3: 88 subjects.
 -fold 4: 88 subjects.
 -fold 5: 88 subjects.


## Sit-and-Stand

In [12]:
train_turn_5f, val_turn_5f, test_turn_5f = [], [], []
train_sitstand_5f, val_sitstand_5f, test_sitstand_5f = [], [], []
train_eyes_5f, val_eyes_5f, test_eyes_5f = [], [], []

def split_5fold_sitstand():
    tmp_train_5f, tmp_val_5f, tmp_test_5f = [], [], []
    for i in range(n_FOLD):
        tmp_train_1f = train_walk_5f[i]
        tmp_val_1f = val_walk_5f[i]
        tmp_test_1f = test_walk_5f[i]
        
        tmp_train_5f.append(tmp_train_1f)
        tmp_val_5f.append(tmp_val_1f)
        tmp_test_5f.append(tmp_test_1f)
    return tmp_train_5f, tmp_val_5f, tmp_test_5f

train_sitstand_5f, val_sitstand_5f, test_sitstand_5f = split_5fold_sitstand()

In [13]:
print(f'[Info] Show Split result of Sit-and-Stand Subtask ...\n')
for i in range(n_FOLD):
    print(f'[Fold {i+1}] Train: (num: {len(train_sitstand_5f[i])})\n{train_sitstand_5f[i]}')
    print(f'[Fold {i+1}] Validate: (num: {len(val_sitstand_5f[i])})\n{val_sitstand_5f[i]}')
    print(f'[Fold {i+1}] Test: (num: {len(test_sitstand_5f[i])})\n{test_sitstand_5f[i]}')    

[Info] Show Split result of Sit-and-Stand Subtask ...

[Fold 1] Train: (num: 56)
['17', '18', '20', '24', '25', '26', '27', '28', '31', '32', '33', '35', '38', '39', '40', '43', '46', '48', '51', '52', '57', '58', '60', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '82', '84', '85', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98', '99']
[Fold 1] Validate: (num: 14)
['03', '05', '06', '07', '09', '10', '100', '12', '13', '16', '19', '21', '22', '30']
[Fold 1] Test: (num: 18)
['01', '02', '04', '11', '29', '34', '36', '37', '44', '45', '49', '59', '67', '75', '81', '83', '89', '94']
[Fold 2] Train: (num: 56)
['17', '18', '20', '24', '25', '26', '27', '28', '30', '31', '32', '33', '34', '36', '37', '38', '39', '40', '44', '45', '46', '48', '49', '51', '57', '58', '59', '60', '63', '64', '67', '68', '69', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '83', '84', '85', '86', '88', '89', '92', 

In [14]:
def check_sitstand_subtask_pid(train_5f, val_5f, test_5f):
    ### check if all subjects are tested
    tmp_set = set()
    for i in range(n_FOLD):
        test_1f = test_5f[i]
        for j in range(len(test_1f)):
            pid = test_1f[j]
            tmp_set.add(pid)
    print(f'Number of pid in Test Set: {len(tmp_set)} subjects.\n')
    
    ### Check each fold (train + val + test)
    print(f'[Info] Check for each fold ...')
    for i in range(n_FOLD):
        tmp_set = set()
        for j in range(len(train_5f[i])):
            tmp_set.add(train_5f[i][j])
        for j in range(len(val_5f[i])):
            tmp_set.add(val_5f[i][j])
        for j in range(len(test_5f[i])):
            tmp_set.add(test_5f[i][j])
        print(f' -fold {i+1}: {len(tmp_set)} subjects.')
    
check_sitstand_subtask_pid(train_sitstand_5f, val_sitstand_5f, test_sitstand_5f)  

Number of pid in Test Set: 88 subjects.

[Info] Check for each fold ...
 -fold 1: 88 subjects.
 -fold 2: 88 subjects.
 -fold 3: 88 subjects.
 -fold 4: 88 subjects.
 -fold 5: 88 subjects.


## Turning

In [15]:
train_turn_5f, val_turn_5f, test_turn_5f = [], [], []

def split_5fold_turn(problem_list):
    tmp_train_5f, tmp_val_5f, tmp_test_5f = [], [], []
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        tmp_train_1f_new, tmp_val_1f_new, tmp_test_1f_new = [], [], []
        
        for j in range(len(train_walk_5f[i])):
            tmp_train_1f.append(train_walk_5f[i][j])
        for j in range(len(val_walk_5f[i])):
            tmp_val_1f.append(val_walk_5f[i][j])
        for j in range(len(test_walk_5f[i])):
            tmp_test_1f.append(test_walk_5f[i][j])

        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            if (pid in problem_list)==False: tmp_train_1f_new.append(pid)
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            if (pid in problem_list)==False: tmp_val_1f_new.append(pid)
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            if (pid in problem_list)==False: tmp_test_1f_new.append(pid)
                
        tmp_train_5f.append(tmp_train_1f_new)
        tmp_val_5f.append(tmp_val_1f_new)
        tmp_test_5f.append(tmp_test_1f_new)
    return tmp_train_5f, tmp_val_5f, tmp_test_5f

train_turn_5f, val_turn_5f, test_turn_5f = split_5fold_turn(Turn_remove_list)

In [16]:
print(f'[Info] Show Split result of Turning Subtask ...\n')
for i in range(n_FOLD):
    print(f'[Fold {i+1}] Train: (num: {len(train_turn_5f[i])})\n{train_turn_5f[i]}')
    print(f'[Fold {i+1}] Validate: (num: {len(val_turn_5f[i])})\n{val_turn_5f[i]}')
    print(f'[Fold {i+1}] Test: (num: {len(test_turn_5f[i])})\n{test_turn_5f[i]}\n')    

[Info] Show Split result of Turning Subtask ...

[Fold 1] Train: (num: 54)
['17', '18', '20', '24', '25', '26', '27', '28', '31', '33', '35', '38', '39', '40', '43', '46', '48', '51', '52', '57', '58', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '82', '84', '85', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98', '99']
[Fold 1] Validate: (num: 12)
['03', '05', '06', '10', '100', '12', '13', '16', '19', '21', '22', '30']
[Fold 1] Test: (num: 17)
['01', '02', '11', '29', '34', '36', '37', '44', '45', '49', '59', '67', '75', '81', '83', '89', '94']

[Fold 2] Train: (num: 54)
['17', '18', '20', '24', '25', '26', '27', '28', '30', '31', '33', '34', '36', '37', '38', '39', '40', '44', '45', '46', '48', '49', '51', '57', '58', '59', '63', '64', '67', '68', '69', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '83', '84', '85', '86', '88', '89', '92', '94', '95', '97', '98', '99']
[Fold 2] Validate

In [17]:
def check_turning_subtask_pid(train_5f, val_5f, test_5f):
    ### check if all subjects are tested
    tmp_set = set()
    for i in range(n_FOLD):
        test_1f = test_5f[i]
        for j in range(len(test_1f)):
            pid = test_1f[j]
            tmp_set.add(pid)
    print(f'Number of pid in Test Set: {len(tmp_set)} subjects.\n')
    
    ### Check each fold (train + val + test)
    print(f'[Info] Check for each fold ...')
    for i in range(n_FOLD):
        tmp_set = set()
        for j in range(len(train_5f[i])):
            tmp_set.add(train_5f[i][j])
        for j in range(len(val_5f[i])):
            tmp_set.add(val_5f[i][j])
        for j in range(len(test_5f[i])):
            tmp_set.add(test_5f[i][j])
        print(f' -fold {i+1}: {len(tmp_set)} subjects.')
    
check_turning_subtask_pid(train_turn_5f, val_turn_5f, test_turn_5f)  

Number of pid in Test Set: 83 subjects.

[Info] Check for each fold ...
 -fold 1: 83 subjects.
 -fold 2: 83 subjects.
 -fold 3: 83 subjects.
 -fold 4: 83 subjects.
 -fold 5: 83 subjects.


## Facial

In [18]:
facial_only_5f = []

def split_facial_only_pid_into_5fold(facial_only_list):
    tmp_5f = []
    
    for i in range(n_FOLD-1):
        pid1 = facial_only_list[i*2]
        pid2 = facial_only_list[i*2+1]
        
        tmp_5f.append([pid1, pid2])
        
    tmp_1f = []
    for i in range(8, len(facial_only_list)):
        pid = facial_only_list[i]
        tmp_1f.append(pid)
    tmp_5f.append(tmp_1f)
    return tmp_5f
    
facial_only_5f = split_facial_only_pid_into_5fold(Eyes_noWalk_list)
facial_only_5f

[['08', '14'], ['15', '23'], ['42', '47'], ['50', '53'], ['54', '55', '56']]

In [19]:
train_eyes_5f, val_eyes_5f, test_eyes_5f = [], [], []

def split_5fold_eyes(onlyEyes, onlyGait, facial_test_5f):
    tmp_train_5f, tmp_val_5f, tmp_test_5f = [], [], []
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        tmp_train_1f_new, tmp_val_1f_new, tmp_test_1f_new = [], [], []
        
        for j in range(len(train_walk_5f[i])):
            tmp_train_1f.append(train_walk_5f[i][j])
        for j in range(len(val_walk_5f[i])):
            tmp_val_1f.append(val_walk_5f[i][j])
        for j in range(len(test_walk_5f[i])):
            tmp_test_1f.append(test_walk_5f[i][j])

        ### Remove pid which does not have facial data
        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            if (pid in onlyGait)==False: tmp_train_1f_new.append(pid)
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            if (pid in onlyGait)==False: tmp_val_1f_new.append(pid)
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            if (pid in onlyGait)==False: tmp_test_1f_new.append(pid)
        
        ### Add pid which only has facial data
        for j in range(len(facial_test_5f[i])):
            pid = facial_test_5f[i][j]
            tmp_test_1f_new.append(pid)
            
        tmp_list_1f = facial_test_5f[i]
        for j in range(len(onlyEyes)):
            pid = onlyEyes[j]
            if pid not in tmp_list_1f:
                tmp_train_1f_new.append(pid)
        
        tmp_train_1f_new.sort()
        tmp_val_1f_new.sort()
        tmp_test_1f_new.sort()
        
        tmp_train_5f.append(tmp_train_1f_new)
        tmp_val_5f.append(tmp_val_1f_new)
        tmp_test_5f.append(tmp_test_1f_new)
    return tmp_train_5f, tmp_val_5f, tmp_test_5f

train_eyes_5f, val_eyes_5f, test_eyes_5f = split_5fold_eyes(Eyes_noWalk_list, Eyes_noEyes_list, facial_only_5f)

In [20]:
print(f'[Info] Show Split result of Facial Subtask ...\n')
for i in range(n_FOLD):
    print(f'[Fold {i+1}] Train: (num: {len(train_eyes_5f[i])})\n{train_eyes_5f[i]}')
    print(f'[Fold {i+1}] Validate: (num: {len(val_eyes_5f[i])})\n{val_eyes_5f[i]}')
    print(f'[Fold {i+1}] Test: (num: {len(test_eyes_5f[i])})\n{test_eyes_5f[i]}\n')    

[Info] Show Split result of Facial Subtask ...

[Fold 1] Train: (num: 62)
['15', '17', '18', '20', '23', '24', '25', '26', '27', '28', '31', '32', '33', '35', '38', '39', '40', '42', '43', '46', '47', '48', '50', '51', '52', '53', '54', '55', '56', '57', '58', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '73', '74', '76', '77', '78', '79', '80', '82', '84', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98', '99']
[Fold 1] Validate: (num: 14)
['03', '05', '06', '07', '09', '10', '100', '12', '13', '16', '19', '21', '22', '30']
[Fold 1] Test: (num: 20)
['01', '02', '04', '08', '11', '14', '29', '34', '36', '37', '44', '45', '49', '59', '67', '75', '81', '83', '89', '94']

[Fold 2] Train: (num: 63)
['08', '14', '17', '18', '20', '24', '25', '26', '27', '28', '30', '31', '32', '33', '34', '36', '37', '38', '39', '40', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '57', '58', '59', '63', '64', '67', '68', '69', '71', '72', '73', 

In [21]:
def check_facial_subtask_pid(train_5f, val_5f, test_5f):
    ### check if all subjects are tested
    tmp_set = set()
    for i in range(n_FOLD):
        test_1f = test_5f[i]
        for j in range(len(test_1f)):
            pid = test_1f[j]
            tmp_set.add(pid)
    print(f'Number of pid in Test Set: {len(tmp_set)} subjects.\n')
    
    ### Check each fold (train + val + test)
    print(f'[Info] Check for each fold ...')
    for i in range(n_FOLD):
        tmp_set = set()
        for j in range(len(train_5f[i])):
            tmp_set.add(train_5f[i][j])
        for j in range(len(val_5f[i])):
            tmp_set.add(val_5f[i][j])
        for j in range(len(test_5f[i])):
            tmp_set.add(test_5f[i][j])
        print(f' -fold {i+1}: {len(tmp_set)} subjects.')
    
check_facial_subtask_pid(train_eyes_5f, val_eyes_5f, test_eyes_5f)  

Number of pid in Test Set: 96 subjects.

[Info] Check for each fold ...
 -fold 1: 96 subjects.
 -fold 2: 96 subjects.
 -fold 3: 96 subjects.
 -fold 4: 96 subjects.
 -fold 5: 96 subjects.


# Show Data Distribution

In [22]:
def show_data_distribution(train_5f, val_5f, test_5f):
    print(f'Data distribution for [Walking Subtask] ...\n')
    
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        
        tmp_train_1f = train_5f[i]
        tmp_val_1f = val_5f[i]
        tmp_test_1f = test_5f[i]
        
        n_AD_train, n_NC_train = 0, 0
        n_AD_val, n_NC_val = 0, 0
        n_AD_test, n_NC_test = 0, 0
        
        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_train += 1
            if label==1: n_AD_train += 1
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_val += 1
            if label==1: n_AD_val += 1
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_test += 1
            if label==1: n_AD_test += 1
            
        print(f'[Fold {i+1}] Total: {n_AD_train+n_AD_val+n_AD_test+n_NC_train+n_NC_val+n_NC_test} subjects.')
        print(f' -(Train) AD: {n_AD_train} subjects, (Val) AD: {n_AD_val} subjects, (Test) AD: {n_AD_test}')
        print(f' -(Train) NC: {n_NC_train} subjects, (Val) NC: {n_NC_val} subjects, (Test) AD: {n_NC_test}')
    
show_data_distribution(train_walk_5f, val_walk_5f, test_walk_5f)

Data distribution for [Walking Subtask] ...

[Fold 1] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 2] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 3] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 4] Total: 88 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 37 subjects, (Val) NC: 10 subjects, (Test) AD: 11
[Fold 5] Total: 88 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 37 subjects, (Val) NC: 10 subjects, (Test) AD: 11


In [23]:
def show_data_distribution(train_5f, val_5f, test_5f):
    print(f'Data distribution for [Sit-and-Stand Subtask] ...\n')
    
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        
        tmp_train_1f = train_5f[i]
        tmp_val_1f = val_5f[i]
        tmp_test_1f = test_5f[i]
        
        n_AD_train, n_NC_train = 0, 0
        n_AD_val, n_NC_val = 0, 0
        n_AD_test, n_NC_test = 0, 0
        
        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_train += 1
            if label==1: n_AD_train += 1
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_val += 1
            if label==1: n_AD_val += 1
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_test += 1
            if label==1: n_AD_test += 1
            
        print(f'[Fold {i+1}] Total: {n_AD_train+n_AD_val+n_AD_test+n_NC_train+n_NC_val+n_NC_test} subjects.')
        print(f' -(Train) AD: {n_AD_train} subjects, (Val) AD: {n_AD_val} subjects, (Test) AD: {n_AD_test}')
        print(f' -(Train) NC: {n_NC_train} subjects, (Val) NC: {n_NC_val} subjects, (Test) AD: {n_NC_test}')

show_data_distribution(train_sitstand_5f, val_sitstand_5f, test_sitstand_5f)

Data distribution for [Sit-and-Stand Subtask] ...

[Fold 1] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 2] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 3] Total: 88 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 4] Total: 88 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 37 subjects, (Val) NC: 10 subjects, (Test) AD: 11
[Fold 5] Total: 88 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 37 subjects, (Val) NC: 10 subjects, (Test) AD: 11


In [24]:
def show_data_distribution(train_5f, val_5f, test_5f):
    print(f'Data distribution for [Turning Subtask] ...\n')
    
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        
        tmp_train_1f = train_5f[i]
        tmp_val_1f = val_5f[i]
        tmp_test_1f = test_5f[i]
        
        n_AD_train, n_NC_train = 0, 0
        n_AD_val, n_NC_val = 0, 0
        n_AD_test, n_NC_test = 0, 0
        
        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_train += 1
            if label==1: n_AD_train += 1
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_val += 1
            if label==1: n_AD_val += 1
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_test += 1
            if label==1: n_AD_test += 1
            
        print(f'[Fold {i+1}] Total: {n_AD_train+n_AD_val+n_AD_test+n_NC_train+n_NC_val+n_NC_test} subjects.')
        print(f' -(Train) AD: {n_AD_train} subjects, (Val) AD: {n_AD_val} subjects, (Test) AD: {n_AD_test}')
        print(f' -(Train) NC: {n_NC_train} subjects, (Val) NC: {n_NC_val} subjects, (Test) AD: {n_NC_test}')
    
show_data_distribution(train_turn_5f, val_turn_5f, test_turn_5f)

Data distribution for [Turning Subtask] ...

[Fold 1] Total: 83 subjects.
 -(Train) AD: 18 subjects, (Val) AD: 3 subjects, (Test) AD: 5
 -(Train) NC: 36 subjects, (Val) NC: 9 subjects, (Test) AD: 12
[Fold 2] Total: 83 subjects.
 -(Train) AD: 18 subjects, (Val) AD: 2 subjects, (Test) AD: 6
 -(Train) NC: 36 subjects, (Val) NC: 9 subjects, (Test) AD: 12
[Fold 3] Total: 83 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 2 subjects, (Test) AD: 5
 -(Train) NC: 36 subjects, (Val) NC: 9 subjects, (Test) AD: 12
[Fold 4] Total: 83 subjects.
 -(Train) AD: 18 subjects, (Val) AD: 3 subjects, (Test) AD: 5
 -(Train) NC: 37 subjects, (Val) NC: 9 subjects, (Test) AD: 11
[Fold 5] Total: 83 subjects.
 -(Train) AD: 17 subjects, (Val) AD: 4 subjects, (Test) AD: 5
 -(Train) NC: 37 subjects, (Val) NC: 10 subjects, (Test) AD: 10


In [25]:
def show_data_distribution(train_5f, val_5f, test_5f):
    print(f'Data distribution for [Facial Subtask] ...\n')
    
    for i in range(n_FOLD):
        tmp_train_1f, tmp_val_1f, tmp_test_1f = [], [], []
        
        tmp_train_1f = train_5f[i]
        tmp_val_1f = val_5f[i]
        tmp_test_1f = test_5f[i]
        
        n_AD_train, n_NC_train = 0, 0
        n_AD_val, n_NC_val = 0, 0
        n_AD_test, n_NC_test = 0, 0
        
        for j in range(len(tmp_train_1f)):
            pid = tmp_train_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_train += 1
            if label==1: n_AD_train += 1
        for j in range(len(tmp_val_1f)):
            pid = tmp_val_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_val += 1
            if label==1: n_AD_val += 1
        for j in range(len(tmp_test_1f)):
            pid = tmp_test_1f[j]
            label = Label_dict[int(pid)]
            if label==0: n_NC_test += 1
            if label==1: n_AD_test += 1
            
        print(f'[Fold {i+1}] Total: {n_AD_train+n_AD_val+n_AD_test+n_NC_train+n_NC_val+n_NC_test} subjects.')
        print(f' -(Train) AD: {n_AD_train} subjects, (Val) AD: {n_AD_val} subjects, (Test) AD: {n_AD_test}')
        print(f' -(Train) NC: {n_NC_train} subjects, (Val) NC: {n_NC_val} subjects, (Test) AD: {n_NC_test}')

show_data_distribution(train_eyes_5f, val_eyes_5f, test_eyes_5f)

Data distribution for [Facial Subtask] ...

[Fold 1] Total: 96 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 43 subjects, (Val) NC: 10 subjects, (Test) AD: 14
[Fold 2] Total: 96 subjects.
 -(Train) AD: 19 subjects, (Val) AD: 4 subjects, (Test) AD: 6
 -(Train) NC: 44 subjects, (Val) NC: 10 subjects, (Test) AD: 13
[Fold 3] Total: 96 subjects.
 -(Train) AD: 20 subjects, (Val) AD: 4 subjects, (Test) AD: 5
 -(Train) NC: 43 subjects, (Val) NC: 10 subjects, (Test) AD: 14
[Fold 4] Total: 96 subjects.
 -(Train) AD: 18 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 45 subjects, (Val) NC: 10 subjects, (Test) AD: 12
[Fold 5] Total: 96 subjects.
 -(Train) AD: 18 subjects, (Val) AD: 5 subjects, (Test) AD: 6
 -(Train) NC: 43 subjects, (Val) NC: 10 subjects, (Test) AD: 14


# Save into CSV file

In [26]:
import csv

def save_5fold_pid_into_csv_file(data, filename):
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)


In [ ]:
save_5fold_pid_into_csv_file(train_walk_5f, 'Training_Fix_5Fold/walk_train_5fold.csv')
save_5fold_pid_into_csv_file(val_walk_5f, 'Training_Fix_5Fold/walk_val_5fold.csv')
save_5fold_pid_into_csv_file(test_walk_5f, 'Training_Fix_5Fold/walk_test_5fold.csv')

In [ ]:
save_5fold_pid_into_csv_file(train_sitstand_5f, 'Training_Fix_5Fold/sitstand_train_5fold.csv')
save_5fold_pid_into_csv_file(val_sitstand_5f, 'Training_Fix_5Fold/sitstand_val_5fold.csv')
save_5fold_pid_into_csv_file(test_sitstand_5f, 'Training_Fix_5Fold/sitstand_test_5fold.csv')

In [ ]:
save_5fold_pid_into_csv_file(train_turn_5f, 'Training_Fix_5Fold/turn_train_5fold.csv')
save_5fold_pid_into_csv_file(val_turn_5f, 'Training_Fix_5Fold/turn_val_5fold.csv')
save_5fold_pid_into_csv_file(test_turn_5f, 'Training_Fix_5Fold/turn_test_5fold.csv')

In [27]:
save_5fold_pid_into_csv_file(train_eyes_5f, 'Training_Fix_5Fold/eyes_train_5fold.csv')
save_5fold_pid_into_csv_file(val_eyes_5f, 'Training_Fix_5Fold/eyes_val_5fold.csv')
save_5fold_pid_into_csv_file(test_eyes_5f, 'Training_Fix_5Fold/eyes_test_5fold.csv')

# (x) Old version

## (x) Load ID

In [43]:
def load_id(dataPath, removeList, mode):
    tmp_list = []
    
    if mode=="Walk":
        print(f'[Info] mode Walk')
        for pid in list(os.listdir(dataPath)):
            if (pid in removeList)==False:
                tmp_list.append(pid)
        print(f'- number of subjects: {len(tmp_list)}')
    elif mode=="Eyes":
        print(f'[Info] mode Eyes')
        for pid in list(os.listdir(dataPath)):
            if (pid in removeList)==False:
                tmp_list.append(pid)
        print(f'- number of subjects: {len(tmp_list)}')
    elif mode=="Sit":
        print(f'[Info] mode Sit')
        for pid in list(os.listdir(dataPath)):
            if (pid in removeList)==False:
                tmp_list.append(pid)
        print(f'- number of subjects: {len(tmp_list)}')
    else:
        print(f'[Info] Something wrong.')
    return tmp_list


walk_remove_list = ["08", "14", "15", "23", "41", "42", "47", "50", "53", "54", "55", "56"]
sit_remove_list = ["08", "14", "15", "23", "41", "42", "47", "50", "53", "54", "55", "56", "16", "26", "51", "59"]
eyes_remove_list = ["10", "11", "41", "60", "61", "85", "96", "97", "98", "99", "100"]
    
idList_walk = load_id(PIDROOT_WALK, walk_remove_list, "Walk")
idList_sit = load_id(PIDROOT_SIT, sit_remove_list, "Sit")
idList_eyes = load_id(PIDROOT_EYES, eyes_remove_list, "Eyes")

[Info] mode Walk
- number of subjects: 88
[Info] mode Sit
- number of subjects: 84
[Info] mode Eyes
- number of subjects: 89


## (x) Split id

In [44]:
def splitID_5Fold(dataPath, pidList, mode):
    tmp_target = []
    trainID_5f, testID_5f = [], []
    
    for pid in pidList:
        # folder = pid_dict[pid]
        # label = Label_dict[int(folder)]
        label = Label_dict[int(pid)]
        tmp_target.append(label)

    kf = StratifiedKFold(n_splits=n_FOLD, shuffle=True, random_state=42)
    
    for train_index, test_index in kf.split(pidList, tmp_target):
        realTrain_idx, realTest_idx = [], []
        for idx in train_index:
            realTrain_idx.append(pidList[idx])
        for idx in test_index:
            realTest_idx.append(pidList[idx])
        
        trainID_5f.append(realTrain_idx)
        testID_5f.append(realTest_idx)
        
    return tmp_target, trainID_5f, testID_5f
    

target_walk, trainID_5f_walk, testID_5f_walk = splitID_5Fold(PIDROOT_WALK, idList_walk, "Walk")
target_sit, trainID_5f_sit, testID_5f_sit = splitID_5Fold(PIDROOT_SIT, idList_sit, "Sit")
target_eyes, trainID_5f_eyes, testID_5f_eyes = splitID_5Fold(PIDROOT_EYES, idList_eyes, "Eyes")

In [45]:
print(f'========== walk ==========')
print(f' -len(target_walk): {len(target_walk)}')
for i in range(n_FOLD):
    print(f' -[Fold {i+1}] len(trainID_5f_walk): {len(trainID_5f_walk[i])}, len(testID_5f_walk): {len(testID_5f_walk[i])}')
    
print(f'======= sit & stand =======')
print(f' -len(target_sit): {len(target_sit)}')
for i in range(n_FOLD):
    print(f' -[Fold {i+1}] len(trainID_5f_sit): {len(trainID_5f_sit[i])}, len(testID_5f_sit): {len(testID_5f_sit[i])}')

print(f'========== eyes ==========')
print(f' -len(target_eyes): {len(target_eyes)}')
for i in range(n_FOLD):
    print(f' -[Fold {i+1}] len(trainID_5f_eyes): {len(trainID_5f_eyes[i])}, len(testID_5f_eyes): {len(testID_5f_eyes[i])}')

========== walk ==========
 -len(target_walk): 88
 -[Fold 1] len(trainID_5f_walk): 70, len(testID_5f_walk): 18
 -[Fold 2] len(trainID_5f_walk): 70, len(testID_5f_walk): 18
 -[Fold 3] len(trainID_5f_walk): 70, len(testID_5f_walk): 18
 -[Fold 4] len(trainID_5f_walk): 71, len(testID_5f_walk): 17
 -[Fold 5] len(trainID_5f_walk): 71, len(testID_5f_walk): 17
======= sit & stand =======
 -len(target_sit): 84
 -[Fold 1] len(trainID_5f_sit): 67, len(testID_5f_sit): 17
 -[Fold 2] len(trainID_5f_sit): 67, len(testID_5f_sit): 17
 -[Fold 3] len(trainID_5f_sit): 67, len(testID_5f_sit): 17
 -[Fold 4] len(trainID_5f_sit): 67, len(testID_5f_sit): 17
 -[Fold 5] len(trainID_5f_sit): 68, len(testID_5f_sit): 16
========== eyes ==========
 -len(target_eyes): 89
 -[Fold 1] len(trainID_5f_eyes): 71, len(testID_5f_eyes): 18
 -[Fold 2] len(trainID_5f_eyes): 71, len(testID_5f_eyes): 18
 -[Fold 3] len(trainID_5f_eyes): 71, len(testID_5f_eyes): 18
 -[Fold 4] len(trainID_5f_eyes): 71, len(testID_5f_eyes): 18
 -[Fol

## (x) Save to CSV

In [46]:
import csv

def write_to_csv(targetList, trainList, testList, fileName1, fileName2, fileName3):
    with open(fileName1, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(targetList)
    with open(fileName2, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(trainList)
    with open(fileName3, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(testList)
    
    
write_to_csv(target_walk, trainID_5f_walk, testID_5f_walk, "walk_target.csv", "walk_train_5fold.csv", "walk_test_5fold.csv")
write_to_csv(target_sit, trainID_5f_sit, testID_5f_sit, "sit_target.csv", "sit_train_5fold.csv", "sit_test_5fold.csv")
write_to_csv(target_eyes, trainID_5f_eyes, testID_5f_eyes, "eyes_target.csv", "eyes_train_5fold.csv", "eyes_test_5fold.csv")